# Restoration

 - Goal: Restore raw detector data, process it and save it for users that want to take home the data!

## Read data batch with sscIO, apply corrections (empty, flatfield, mask) and restore data

In [9]:
from sscCdi.caterete.ptycho_restauration import restauration_processing_binning
import json
import numpy as np

def read_diffraction_patterns(path,jason):

    os.system(f"h5clear -s {path}") # gambiarra because file is not closed at the backend!
    os.system(f"h5clear -s {jason['EmptyFrame']}") # gambiarra because file is not closed at the backend!

    """ Read raw data """
    raw_difpads,_ = io.read_volume(path, 'numpy', use_MPI=True, nprocs=jason["Threads"])

    """ Get inputs from json file """
    z1 = float(jason["DetDistance"]) * 1000  # Here comes the distance Geometry(Z1):

    empty = np.asarray(h5py.File(jason['EmptyFrame'], 'r')['/entry/data/data']).squeeze().astype(np.float32)

    flat = np.array(h5py.File(jason["FlatField"], 'r')['entry/data/data'][()][0, 0, :, :])
    flat[np.isnan(flat)] = -1
    flat[flat == 0] = 1

    mask = h5py.File(jason["Mask"], 'r')['entry/data/data'][()][0, 0, :, :]

    centerx, centery = center
    hsize = jason['DetectorROI']   
    Binning = int(jason['Binning'])

    """ Get detector geometry from distance """
    geometry = Geometry(z1)

    """ Call corrections and restoration """
    r_params = (Binning, empty, flat, centerx, centery, hsize, geometry, mask, jason)
    output, _ = pi540D.backward540D_nonplanar_batch(raw_difpads, z1, jason['Threads'], [ hsize//2 , hsize//2 ], restauration_processing_binning,  r_params, 'only') # Apply empty, flatfield, mask and restore!

    """ How the correction are made prior to restoration:
    
        img[empty > 1] = -1 # Apply empty 
        img = img * np.squeeze(flat) # Apply flatfield
        img[np.abs(mask) ==1] = -1   # Apply Mask
        img = img[cy - hsize:cy + hsize, cx - hsize:cx + hsize] # Center data

    """
    return output


### Inputs

In [ ]:
jason = json.load(open("ptycho_input_json_example.json"))
path = "path/to/diffraction_pattern/file.hdf5"
path_to_save = "path/to/output/savefile.hdf5"

### Call function

In [ ]:
processed_data = read_diffraction_patterns(path,jason)

## Save corrected+restored data in hdf5 for the user

In [10]:
#TODO: implement save function